# PD controller of helicopter motion from hover

The following scripts runs a simulation of helicopter motion with a PD controller. The first cell gets the data and all other cells create relevant plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# INITIAL DATA HELICOPTER
g = 9.81
cla = 5.7  # NACA 0012
volh = 0.075  # blade solidity
lok = 6
cds = 1.5
mass = 2200
rho = 1.225
vtip = 200
diam = 2 * 7.32
iy = 10615
mast = 1
omega = vtip / (diam / 2)
area = np.pi / 4 * diam**2
tau = 0.1  # time constant in dynamics inflow!!!
collect = np.array([6 * np.pi / 180])
longit = np.array([0 * np.pi / 180])

# initial values
t0 = 0
u0 = 0
w0 = 0
q0 = 0
pitch0 = 0 * np.pi / 180
x0 = 0
labi0 = np.sqrt(mass * g / (area * 2 * rho)) / vtip

t = [t0]
u = [u0]
w = [w0]
q = [q0]
pitch = [pitch0]
x = [x0]
labi = [labi0]
z = [0]

# INTEGRATION
aantal = 800
teind = 80
stap = (teind - t0) / aantal

for i in range(aantal):
    ti = t[-1]
    if 0.5 <= ti < 1:
        longit = np.append(longit, 1 * np.pi / 180)
    else:
        longit = np.append(longit, 0 * np.pi / 180)

    if ti >= 15:
        longitgrd = 0.2 * pitch[-1] * 180 / np.pi + 0.2 * q[-1] * 180 / np.pi  # PD in deg
        longit[-1] = longitgrd * np.pi / 180  # in rad

    # NO LAW FOR COLLECTIVE
    c = u[-1] * np.sin(pitch[-1]) - w[-1] * np.cos(pitch[-1])
    h = -z[-1]
    collect = np.append(collect, collect[0])

    # Defining the differential equations
    qdiml = q[-1] / omega
    vdiml = np.sqrt(u[-1]**2 + w[-1]**2) / vtip
    if u[-1] == 0:
        phi = np.pi / 2 if w[-1] > 0 else -np.pi / 2
    else:
        phi = np.arctan(w[-1] / u[-1])
    if u[-1] < 0:
        phi += np.pi
    alfc = longit[-1] - phi

    mu = vdiml * np.cos(alfc)
    labc = vdiml * np.sin(alfc)

    # a1 Flapping calculi
    teller = -16 / lok * qdiml + 8 / 3 * mu * collect[-1] - 2 * mu * (labc + labi[-1])
    a1 = teller / (1 - .5 * mu**2)

    # the thrust coefficient
    ctelem = cla * volh / 4 * (2 / 3 * collect[-1] * (1 + 1.5 * mu**2) - (labc + labi[-1]))
    # Thrust coefficient from Glauert
    alfd = alfc - a1
    ctglau = 2 * labi[-1] * np.sqrt((vdiml * np.cos(alfd))**2 + (vdiml * np.sin(alfd) + labi[-1])**2)

    # Equations of motion
    labidot = ctelem
    thrust = labidot * rho * vtip**2 * area
    helling = longit[-1] - a1
    vv = vdiml * vtip  # it is 1/sqrt(u^2+w^2)

    udot = -g * np.sin(pitch[-1]) - cds / mass * .5 * rho * u[-1] * vv + thrust / mass * np.sin(helling) - q[-1] * w[-1]
    wdot = g * np.cos(pitch[-1]) - cds / mass * .5 * rho * w[-1] * vv - thrust / mass * np.cos(helling) + q[-1] * u[-1]
    qdot = -thrust * mast / iy * np.sin(helling)
    pitchdot = q[-1]

    xdot = u[-1] * np.cos(pitch[-1]) + w[-1] * np.sin(pitch[-1])
    zdot = -c
    labidot = (ctelem - ctglau) / tau

    u.append(u[-1] + stap * udot)
    w.append(w[-1] + stap * wdot)
    q.append(q[-1] + stap * qdot)
    pitch.append(pitch[-1] + stap * pitchdot)
    x.append(x[-1] + stap * xdot)
    labi.append(labi[-1] + stap * labidot)
    z.append(z[-1] + stap * zdot)
    t.append(ti + stap)

# Let us now plot the results

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=u, mode='lines', name='u(m/s)'))
fig.update_layout(title='Velocity (u) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='u(m/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.array(pitch) * 180 / np.pi, mode='lines', name='pitch(deg)'))
fig.update_layout(title='Pitch over Time',
                  xaxis_title='t (s)',
                  yaxis_title='pitch(deg)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=x, mode='lines', name='x(m)'))
fig.update_layout(title='Displacement in X-Direction over Time',
                  xaxis_title='t (s)',
                  yaxis_title='x(m)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=w, mode='lines', name='w(m/s)'))
fig.update_layout(title='Vertical Velocity (w) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='w(m/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=q, mode='lines', name='q(rad/s)'))
fig.update_layout(title='Rate of Pitch Change (q) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='q(rad/s)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=labi, mode='lines', name='labi'))
fig.update_layout(title='Non-Dimensional Inflow (labi) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='labi')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=-np.array(z), mode='lines', name='h(m)'))
fig.update_layout(title='Altitude (h) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='h(m)')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=longit * 180 / np.pi, mode='lines', name='longit(deg)'))
fig.update_layout(title='Longitudinal Input (longit) over Time',
                  xaxis_title='t (s)',
                  yaxis_title='longit(deg)')
fig.show()
